<a href="https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/ViLT/Fine_tuning_ViLT_for_VQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning ViLT For visual question answering (VQA)

In this notebook, we are going to illustrate how to fine-tune the Vision-and-Language Transformer (ViLT) for visual question answering. This is going to be very similar to how one would fine-tune BERT: one just places a head on top that is randomly initialized, and trains it end-to-end together with a pre-trained base.

* Paper: https://arxiv.org/abs/2102.03334
* ViLT docs: https://huggingface.co/docs/transformers/master/en/model_doc/vilt


## Set-up environment

First, we install the Transformers library.

In [1]:
%pip install -q transformers

Note: you may need to restart the kernel to use updated packages.


## Load data

Next, we load the data. The data of VQAv2 can be obtained from the [official website](https://visualqa.org/download.html).

For demonstration purposes, we only download the validation dataset. We download:
* the images (stored in a single folder)
* the questions (stored in a JSON)
* the annotations (stored in a JSON) a.k.a. the answers to the questions.

I've downloaded the images myself, and stored them locally.

### Read questions

First, we read the questions.

In [3]:
from datasets import load_dataset

dataset = load_dataset("HuggingFaceM4/VQAv2")

/home/mjschock/Projects/mjschock/timestep/src/timestep/services/backend/app/workflows/agents/starcoder/.venv/lib/python3.11/site-packages/datasets/load.py:1429: FutureWarning: The repository for HuggingFaceM4/VQAv2 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/HuggingFaceM4/VQAv2
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


/home/mjschock/Projects/mjschock/timestep/src/timestep/services/backend/app/workflows/agents/starcoder/.venv/lib/python3.11/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


KeyboardInterrupt: 

In [2]:
import json

# Opening JSON file
f = open('/content/drive/MyDrive/ViLT/Datasets/VQAv2/v2_OpenEnded_mscoco_val2014_questions.json')

# Return JSON object as dictionary
data_questions = json.load(f)
print(data_questions.keys())

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ViLT/Datasets/VQAv2/v2_OpenEnded_mscoco_val2014_questions.json'

Let's see how many questions there are:

In [ ]:
questions = data_questions['questions']
print("Number of questions:", len(questions))

That's quite a lot! Let's take a look at the first one:

In [ ]:
questions[0]

As we can see, this question is related to an image with a certain ID. How can we find back which image this is? The function below allows to get the ID from a corresponding filename. We'll use it to map between image IDs and their corresponding filenames.

In [ ]:
import re
from typing import Optional

filename_re = re.compile(r".*(\d{12})\.((jpg)|(png))")

# source: https://github.com/allenai/allennlp-models/blob/a36aed540e605c4293c25f73d6674071ca9edfc3/allennlp_models/vision/dataset_readers/vqav2.py#L141
def id_from_filename(filename: str) -> Optional[int]:
    match = filename_re.fullmatch(filename)
    if match is None:
        return None
    return int(match.group(1))

In [ ]:
from os import listdir
from os.path import isfile, join
from tqdm.auto import tqdm

# root at which all images are stored
root = '/content/drive/MyDrive/ViLT/Datasets/VQAv2/val2014'
file_names = [f for f in tqdm(listdir(root)) if isfile(join(root, f))]

We can map a filename to its ID as follows:

In [ ]:
id_from_filename('COCO_val2014_000000501080.jpg')

We create 2 dictionaries, one that maps filenames to their IDs and one the other way around:

In [ ]:
filename_to_id = {root + "/" + file: id_from_filename(file) for file in file_names}
id_to_filename = {v:k for k,v in filename_to_id.items()}

We can now find back the image to which the question 'Where is he looking?' corresponded:

In [ ]:
from PIL import Image

path = id_to_filename[questions[0]['image_id']]
image = Image.open(path)
image

### Read annotations

Next, let's read the annotations. As we'll see, every image is annotated with multiple possible answers.

In [ ]:
import json

# Read annotations
f = open('/content/drive/MyDrive/ViLT/Datasets/VQAv2/v2_mscoco_val2014_annotations.json')

# Return JSON object as dictionary
data_annotations = json.load(f)
print(data_annotations.keys())

As we can see, there are 214354 annotations in total (for the validation dataset only!).

In [ ]:
annotations = data_annotations['annotations']

In [ ]:
print("Number of annotations:", len(annotations))

Let's take a look at the first one. As we can see, the example contains several answers (collected by different human annotators). The answer to a question can be a bit subjective: for instance for the question "where is he looking?", some people annotated this with "down", others with "table", another one with "skateboard", etc. So there's a bit of disambiguity among the annotators :)

In [ ]:
annotations[0]

## Add labels + scores

Due to this ambiguity, most authors treat the VQAv2 dataset as a multi-label classification problem (as multiple answers are possibly valid). Moreover, rather than just creating a one-hot encoded vector, one creates a soft encoding, based on the number of times a certain answer appeared in the annotations.

For instance, in the example above, the answer "down" seems to be selected way more often than "skateboard". Hence, we want the model to give more emphasis on "down" than on "skateboard". We can achieve this by giving a score of 1.0 to labels which are counted at least 3 times, and a score < 1.0 for labels that are counted less.

We'll add 2 keys to each annotations:
* labels, which is a list of integer indices of the labels that apply to a given image + question.
* scores, which are the corresponding scores (between 0 and 1), which indicate the importance of each label.

As we'll need the id2label mapping from the VQA dataset, we load it from the hub as follows:

In [ ]:
from transformers import ViltConfig

config = ViltConfig.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

In [ ]:
from tqdm.notebook import tqdm

def get_score(count: int) -> float:
    return min(1.0, count / 3)

for annotation in tqdm(annotations):
    answers = annotation['answers']
    answer_count = {}
    for answer in answers:
        answer_ = answer["answer"]
        answer_count[answer_] = answer_count.get(answer_, 0) + 1
    labels = []
    scores = []
    for answer in answer_count:
        if answer not in list(config.label2id.keys()):
            continue
        labels.append(config.label2id[answer])
        score = get_score(answer_count[answer])
        scores.append(score)
    annotation['labels'] = labels
    annotation['scores'] = scores

Let's verify an example:

In [ ]:
annotations[0]

Let's verify the labels and corresponding scores:

In [ ]:
labels = annotations[0]['labels']
print([config.id2label[label] for label in labels])

In [ ]:
scores = annotations[0]['scores']
print(scores)

## Create PyTorch dataset

Next, we create a regular [PyTorch dataset](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html). We leverage `ViltProcessor` to prepare each image + text pair for the model, which will automatically:
* leverage `BertTokenizerFast` to tokenize the text and create `input_ids`, `attention_mask` and `token_type_ids`
* leverage `ViltImageProcessor` to resize + normalize the image and create `pixel_values` and `pixel_mask`.

Note that the docs of `ViltProcessor` can be found [here](https://huggingface.co/docs/transformers/master/en/model_doc/vilt#transformers.ViltProcessor).

We also add the labels. This is a PyTorch tensor of shape `(num_labels,)` that contains the soft encoded vector.

In [ ]:
import torch
from PIL import Image

class VQADataset(torch.utils.data.Dataset):
    """VQA (v2) dataset."""

    def __init__(self, questions, annotations, processor):
        self.questions = questions
        self.annotations = annotations
        self.processor = processor

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        # get image + text
        annotation = self.annotations[idx]
        questions = self.questions[idx]
        image = Image.open(id_to_filename[annotation['image_id']])
        text = questions['question']

        encoding = self.processor(image, text, padding="max_length", truncation=True, return_tensors="pt")
        # remove batch dimension
        for k,v in encoding.items():
          encoding[k] = v.squeeze()
        # add labels
        labels = annotation['labels']
        scores = annotation['scores']
        # based on: https://github.com/dandelin/ViLT/blob/762fd3975c180db6fc88f577cf39549983fa373a/vilt/modules/objectives.py#L301
        targets = torch.zeros(len(config.id2label))
        for label, score in zip(labels, scores):
              targets[label] = score
        encoding["labels"] = targets

        return encoding

In [ ]:
from transformers import ViltProcessor

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-mlm")

dataset = VQADataset(questions=questions[:100],
                     annotations=annotations[:100],
                     processor=processor)

In [ ]:
dataset[0].keys()

In [ ]:
processor.decode(dataset[0]['input_ids'])

In [ ]:
labels = torch.nonzero(dataset[0]['labels']).squeeze().tolist()

In [ ]:
[config.id2label[label] for label in labels]

## Define model

Here we define a `ViltForQuestionAnswering` model, with the weights of the body initialized from dandelin/vilt-b32-mlm, and a randomly initialized classification head. We also move it to the GPU, if it's available.

In [ ]:
from transformers import ViltForQuestionAnswering

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-mlm",
                                                 id2label=config.id2label,
                                                 label2id=config.label2id)
model.to(device)

Next, we create a corresponding PyTorch DataLoader, which allows us to iterate over the dataset in batches.

Due to the fact that the processor resizes images to not necessarily the same size, we leverage the `pad_and_create_pixel_mask` method of the processor to pad the pixel values of a batch and create a corresponding pixel mask, which is a tensor of shape (batch_size, height, width) indicating which pixels are real (1) and which are padding (0).

In [ ]:
from torch.utils.data import DataLoader

def collate_fn(batch):
  input_ids = [item['input_ids'] for item in batch]
  pixel_values = [item['pixel_values'] for item in batch]
  attention_mask = [item['attention_mask'] for item in batch]
  token_type_ids = [item['token_type_ids'] for item in batch]
  labels = [item['labels'] for item in batch]

  # create padded pixel values and corresponding pixel mask
  encoding = processor.image_processor.pad(pixel_values, return_tensors="pt")

  # create new batch
  batch = {}
  batch['input_ids'] = torch.stack(input_ids)
  batch['attention_mask'] = torch.stack(attention_mask)
  batch['token_type_ids'] = torch.stack(token_type_ids)
  batch['pixel_values'] = encoding['pixel_values']
  batch['pixel_mask'] = encoding['pixel_mask']
  batch['labels'] = torch.stack(labels)

  return batch

train_dataloader = DataLoader(dataset, collate_fn=collate_fn, batch_size=4, shuffle=True)

Let's verify a batch:

In [ ]:
batch = next(iter(train_dataloader))

In [ ]:
for k,v in batch.items():
  print(k, v.shape)

We can verify a given training example, by visualizing the image:

In [ ]:
from PIL import Image
import numpy as np

image_mean = processor.image_processor.image_mean
image_std = processor.image_processor.image_std

batch_idx = 1

unnormalized_image = (batch["pixel_values"][batch_idx].numpy() * np.array(image_mean)[:, None, None]) + np.array(image_std)[:, None, None]
unnormalized_image = (unnormalized_image * 255).astype(np.uint8)
unnormalized_image = np.moveaxis(unnormalized_image, 0, -1)
Image.fromarray(unnormalized_image)

In [ ]:
processor.decode(batch["input_ids"][batch_idx])

In [ ]:
labels = torch.nonzero(batch['labels'][batch_idx]).squeeze().tolist()

In [ ]:
[config.id2label[label] for label in labels]

## Train a model

Finally, let's train a model! Note that I haven't done any hyperparameter tuning as this notebook was just created for demo purposes. I'd recommend going over the [ViLT paper](https://arxiv.org/abs/2102.03334) for better training settings.

I just wanted to illustrate that you can make the model overfit this dataset.

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

model.train()
for epoch in range(50):  # loop over the dataset multiple times
   print(f"Epoch: {epoch}")
   for batch in tqdm(train_dataloader):
        # get the inputs;
        batch = {k:v.to(device) for k,v in batch.items()}

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(**batch)
        loss = outputs.loss
        print("Loss:", loss.item())
        loss.backward()
        optimizer.step()

## Inference

Let's verify whether the model has actually learned something:

In [ ]:
example = dataset[0]
print(example.keys())

In [ ]:
processor.decode(example['input_ids'])

In [ ]:
# add batch dimension + move to GPU
example = {k: v.unsqueeze(0).to(device) for k,v in example.items()}

# forward pass
outputs = model(**example)

Note that we need to apply a sigmoid activation on the logits since the model is trained using binary cross-entropy loss (as it frames VQA as a multi-label classification task).

In [ ]:
logits = outputs.logits
predicted_classes = torch.sigmoid(logits)

probs, classes = torch.topk(predicted_classes, 5)

for prob, class_idx in zip(probs.squeeze().tolist(), classes.squeeze().tolist()):
  print(prob, model.config.id2label[class_idx])